In [4]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langgraph.checkpoint.memory import InMemorySaver

In [5]:
load_dotenv()
llm = ChatOpenAI()

In [6]:

class JokeState(TypedDict):
    topic: str
    joke: str
    explanation: str

In [7]:
def generate_joke(state: JokeState):
    prompt = f'write few line joke about the topic - {state['topic']}'
    
    response = llm.invoke(prompt).content
    
    return {'joke': response}

def generate_explanation(state: JokeState):
    prompt = f'write an explanation for the joke - {state['joke']}'
    
    response = llm.invoke(prompt).content
    
    return {'explanation': response}



In [8]:
graph = StateGraph(JokeState)

graph.add_node('generate_joke', generate_joke)
graph.add_node('generate_explanation', generate_explanation)

graph.add_edge(START, 'generate_joke')
graph.add_edge('generate_joke', 'generate_explanation')
graph.add_edge('generate_explanation', END)


checkpointer = InMemorySaver()

workflow = graph.compile(checkpointer=checkpointer)


In [14]:
config1 = {'configurable': {"thread_id": "1"}}

workflow.invoke({'topic': 'sandwitch'}, config=config1)

{'topic': 'sandwitch',
 'joke': 'Why did the sandwich go to the dentist? Because it had a filling!',
 'explanation': 'This joke plays on the double meaning of the word "filling." In dentistry, a filling is a material used to repair a cavity in a tooth. However, in the context of a sandwich, a filling refers to the tasty ingredients that are placed between the bread. So, when the joke says the sandwich went to the dentist because it had a filling, it is meant to be a play on words and not to be taken literally. It is a lighthearted and silly way to humorously explain why the sandwich needed to see the dentist.'}

In [16]:
workflow.get_state(config1)

StateSnapshot(values={'topic': 'sandwitch', 'joke': 'Why did the sandwich go to the dentist? Because it had a filling!', 'explanation': 'This joke plays on the double meaning of the word "filling." In dentistry, a filling is a material used to repair a cavity in a tooth. However, in the context of a sandwich, a filling refers to the tasty ingredients that are placed between the bread. So, when the joke says the sandwich went to the dentist because it had a filling, it is meant to be a play on words and not to be taken literally. It is a lighthearted and silly way to humorously explain why the sandwich needed to see the dentist.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f070a15-ac39-6ac1-8006-eaf746035ebc'}}, metadata={'source': 'loop', 'writes': {'generate_explanation': {'explanation': 'This joke plays on the double meaning of the word "filling." In dentistry, a filling is a material used to repair a cavity in a tooth. However, in th

In [17]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'sandwitch', 'joke': 'Why did the sandwich go to the dentist? Because it had a filling!', 'explanation': 'This joke plays on the double meaning of the word "filling." In dentistry, a filling is a material used to repair a cavity in a tooth. However, in the context of a sandwich, a filling refers to the tasty ingredients that are placed between the bread. So, when the joke says the sandwich went to the dentist because it had a filling, it is meant to be a play on words and not to be taken literally. It is a lighthearted and silly way to humorously explain why the sandwich needed to see the dentist.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f070a15-ac39-6ac1-8006-eaf746035ebc'}}, metadata={'source': 'loop', 'writes': {'generate_explanation': {'explanation': 'This joke plays on the double meaning of the word "filling." In dentistry, a filling is a material used to repair a cavity in a tooth. However, in t

## Time Travel



In [ ]:
## Get to that specific checkpoint you want to go to.
workflow.get_state({'configurable': {"thread_id": "1", 'checkpoint_id': '1f070a0f-8e69-6058-8000-80892ddec361'}})

StateSnapshot(values={'topic': 'pizza'}, next=('generate_joke',), config={'configurable': {'thread_id': '1', 'checkpoint_id': '1f070a0f-8e69-6058-8000-80892ddec361'}}, metadata={'source': 'loop', 'writes': None, 'thread_id': '1', 'step': 0, 'parents': {}}, created_at='2025-08-03T19:35:12.142292+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f070a0f-8e60-6d6a-bfff-6011f6a4e732'}}, tasks=(PregelTask(id='4ee39e55-7508-c0e7-fcd7-cef8ca8b8821', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error=None, interrupts=(), state=None, result={'joke': 'Why did the pizza go to the party? Because it wanted to be the "life of the party"!'}),))

In [19]:
workflow.invoke(None, {'configurable': {"thread_id": "1", 'checkpoint_id': '1f070a0f-8e69-6058-8000-80892ddec361'}})

{'topic': 'pizza',
 'joke': 'Why did the pizza go to the party? Because it wanted to get a little saucy!\n\nI ordered a pizza with extra cheese and it was grate!\n\nWhy did the pizza maker go broke? Because he kneaded dough!',
 'explanation': 'This joke is a play on words with the term "saucy", which can mean behaving in a bold or cheeky manner, as well as referring to pizza sauce. The punchline suggests that the pizza went to the party in order to act a little bold or cheeky, but also plays on the fact that pizza sauce is a key ingredient in pizza. Overall, the joke combines both literal and figurative interpretations of the word "saucy" for a humorous effect.'}

In [20]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza go to the party? Because it wanted to get a little saucy!\n\nI ordered a pizza with extra cheese and it was grate!\n\nWhy did the pizza maker go broke? Because he kneaded dough!', 'explanation': 'This joke is a play on words with the term "saucy", which can mean behaving in a bold or cheeky manner, as well as referring to pizza sauce. The punchline suggests that the pizza went to the party in order to act a little bold or cheeky, but also plays on the fact that pizza sauce is a key ingredient in pizza. Overall, the joke combines both literal and figurative interpretations of the word "saucy" for a humorous effect.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f070a35-7298-6b0e-8002-3c539dfdd662'}}, metadata={'source': 'loop', 'writes': {'generate_explanation': {'explanation': 'This joke is a play on words with the term "saucy", which can mean behaving in a bold or cheeky

## Update State

In [26]:
workflow.update_state({'configurable': {"thread_id": "1", 'checkpoint_id': '1f070a0f-8e69-6058-8000-80892ddec361', 'checkpoint_ns': ""}}, {'topic': 'samosa'})

{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1f070a45-a6f5-6a3f-8001-fad962ee2d67'}}

In [28]:
workflow.invoke(None, {'configurable': {"thread_id": "1", 'checkpoint_id': '1f070a45-a6f5-6a3f-8001-fad962ee2d67'}})

{'topic': 'samosa',
 'joke': 'Why did the samosa go to therapy? Because it had too many layers to unpack!',
 'explanation': 'This joke plays on the idea of a samosa, a popular Indian snack with multiple layers of crispy dough filled with spiced potatoes and other ingredients. In therapy, patients often talk about their feelings and experiences, metaphorically "unpacking" their emotions and thoughts to better understand themselves. The joke humorously suggests that the samosa went to therapy because it had too many layers (literally) to unpack, implying that the snack itself was feeling overwhelmed by its own complexity.'}

In [29]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'samosa', 'joke': 'Why did the samosa go to therapy? Because it had too many layers to unpack!', 'explanation': 'This joke plays on the idea of a samosa, a popular Indian snack with multiple layers of crispy dough filled with spiced potatoes and other ingredients. In therapy, patients often talk about their feelings and experiences, metaphorically "unpacking" their emotions and thoughts to better understand themselves. The joke humorously suggests that the samosa went to therapy because it had too many layers (literally) to unpack, implying that the snack itself was feeling overwhelmed by its own complexity.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f070a46-65d0-6dd3-8003-f4df18810444'}}, metadata={'source': 'loop', 'writes': {'generate_explanation': {'explanation': 'This joke plays on the idea of a samosa, a popular Indian snack with multiple layers of crispy dough filled with spiced potatoes and othe